# A02. MLB API
This extracts DraftKings contest data and saves results
- Type: Data
- Run Frequency: Once daily
- Sources:
    - MLB Stats API
    - Statcast (via pybaseball package)
- Dates:
    - Created: 9/23/2023
    - Updated: 4/21/2024

### Imports

In [1]:
import sys
if not hasattr(sys.modules['__main__'], '__file__'):
    %run "C:\Users\james\Documents\MLB\Code\U1. Imports.ipynb"
    %run "C:\Users\james\Documents\MLB\Code\U2. Utilities.ipynb"
    %run "C:\Users\james\Documents\MLB\Code\U3. Classes.ipynb"
    %run "C:\Users\james\Documents\MLB\Code\U4. Datasets.ipynb"
    print("Imports executed")

C:\Users\james\anaconda3\lib\site-packages\pybaseball\statcast.py
Imports executed


### Settings

In [2]:
if not hasattr(sys.modules['__main__'], '__file__'):
    year = 2025
    print("Local settings")

Local settings


### Run

Note: functions have all been moved to U4. Datasets

##### 1. Stats API 

In [ ]:
statsapi_df = plays_statsapi(f"03/18/{year}", f"11/15/{year}")
statsapi_df.to_csv(os.path.join(baseball_path, "A02. MLB API", "1. Stats API", f"Stats API {year}.csv"), index=False, encoding='iso-8859-1')

del statsapi_df

##### 2. Statcast

In [5]:
%%time
# statcast_df = plays_statcast(f"{year}-03-18", f"{year}-11-15") # This randomly started generating errors on both ends of range
statcast_df = plays_statcast(f"{year}-03-18", yesterdaysdate_dash)
statcast_df.to_csv(os.path.join(baseball_path, "A02. MLB API", "2. Statcast", f"Statcast {year}.csv"), index=False, encoding='iso-8859-1')

del statcast_df

This is a large query, it may take a moment to complete


100%|██████████| 66/66 [02:10<00:00,  1.97s/it]


CPU times: total: 3min 47s
Wall time: 2min 14s
